<a href="https://colab.research.google.com/github/LFGuzman2105/proyecto_sp1_IA/blob/master/proyecto_IA_dermatologia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>